In [ ]:
# Response_evaluation_mk15

Script to evaluate engine response or LR

Daniel T 2017-02-08

Changelog

17-06-05 - changed import to automaticly replace ECU --> no excel prep needed.
           this is based on that the file is configured with param/desc/unit/data...

17-05-16 - changed t60 -> t50

17-04-10 - add if statement to fix if 90% tc_trq > max(trqdyn)

Timer for time measurement

In [ ]:
import time 
startTime = time.time()

Import necessary libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline  

Find folder and file(s)

In [ ]:
FilePath = 'C:\Dokument - DT\Python\Respons\Measdata' # Specify filepath where CSV files are located
ResultPath = 'C:\Dokument - DT\Python\Respons\Resultat' # Specfy where to put excel files

FileNames = [f for f in os.listdir(FilePath) if f.lower().endswith('.txt')] #Specify Filenames
print (FilePath)
print ('\nFile/s in folder\n')
for i in enumerate(FileNames):
    print(i[0], i[1])

Choose which file to run eval on

In [ ]:
fileno = 0

GTT = False

ImportFile = os.path.join(FilePath, FileNames[fileno]) # Add path + name
print(ImportFile)

In [ ]:
SignalList = ["Spd_Engine",
              "Trq_Engine",
              "S_Pos_Throttle",
              "Conc_Soot483",
              "Conc_NOx",
              "se_NOxInLevel",
              "tc_TorqueValue",
              "Spd_Turbo_rpm",
              "P_Cyl_Peak"]

#Specify limits for plots
EngineSpeed = [500, 2000, 250]
EngineTrq = [-400, 3000, 200]
TurboSpeed = [25000,125000,25000]
Pcyl = [8,22,1]

Measurement setup

In [ ]:
dt = 0.1 #10hz = 0.1, 25hz = 0.04 #time sample in meas

# create variables
# max target for time2torque
time2trq = (np.zeros((4,3)))
time2trq[:,0] = [500, 1000, 1500, 2500]
time2trq[:,1] = [2.5, 2.5, 1.5, 1.5]
time2trq[:,2] = [3.3, 3.3, 2, 2] 
time2trq = pd.DataFrame(data=time2trq)
time2trq = time2trq.rename(columns={0: 'Engine_Speed', 1:'T_75_target', 2:'T_90_Target'})
time2trq

Functions


In [ ]:
# function to linear interpolate time asf dt in two given points
def linear_interp(array,value):
    interp_axis = [0, dt] # create x-axis (time)
    interp_points = [min((np.where(array > value)[0]))-1, min((np.where(array > value)[0]))] # find points below and above value
    interp_values = [np.array(array)[interp_points[0]], np.array(array)[interp_points[1]]] # find measured points according to points above
    interp_result = np.interp(value,interp_values,interp_axis) # interpolate, result is in dt
    return  interp_result

Import file

In [ ]:
df_header = pd.read_csv(ImportFile, sep='\t', low_memory=False, nrows=2) # header/desc and units with pandas
df_header.head()

In [ ]:
df_head_ECU = df_header #save original header with ECU in name

In [ ]:
filter_col = [col for col in list(df_header) if col.startswith('ECU')] # find how many ECU parameters are in mes file

In [ ]:
for filter_no in filter_col: # replace ECU with names
    df_header = df_header.rename(columns = {filter_no:df_header[filter_no][0]})

In [ ]:
header_list = df_header.columns #create a list of all available parameters

Find index of each parameter in file

In [ ]:
usecols = []
for Signal in SignalList:
    temp_index = np.where(header_list == Signal)[0][0]
    usecols.append(temp_index)

In [ ]:
usecols #index of each signal to use with import of all data

In [ ]:
df= pd.read_csv(ImportFile, sep='\t', low_memory=False, skiprows=[1,2], usecols=usecols) #import data with usecols
df.head()

In [ ]:
header_list2 = df.columns # create a second list to find which ECUs are imported

In [ ]:
filter_col2 = list(set(header_list2)-set(SignalList)) # filter out ECU names only
filter_col2

In [ ]:
for filter_no in filter_col2: # rename ECU names in final import
    df = df.rename(columns = {filter_no:df_head_ECU[filter_no][0]})
    
df.head()

In [ ]:
# plot enginespeed and torque
fig, ax1 = plt.subplots()

ax1.set_title('Speed/Torque')

ax1.plot(df.Spd_Engine[2:], 'b-')
ax1.set_xlabel('Sample')
ax1.set_ylim([500,2500])
ax1.set_yticks(np.arange(500,2750,250))
ax1.set_ylabel('Spd_Engine')


ax2 = ax1.twinx()
ax2.plot(df.Trq_Engine[2:], 'r-')
ax2.set_ylim([0,max(df.Trq_Engine)+200])
ax2.set_yticks(np.arange(0,1000,250))
ax2.set_ylabel('Trq_Engine')

plt.show()

In [ ]:
# find where the response start/stop

# engine respone is started when trhottle go from x% to 100%
throttlechange = np.append(np.diff(df.S_Pos_Throttle),0)#lägg till 0 på slutet

start = list(np.where(throttlechange > 40)[0])
start_check = list(np.where(throttlechange < -5)[0])
print(start,'\n', start_check)

plt.plot(throttlechange)
plt.show

In [ ]:
# statement to clean out dirty data

if (start < start_check):
    print('start 0 adjusted')
    start = start[1:]
    print(start)
elif (df.Spd_Engine[0] < df.Spd_Engine[start[0]]):
    print('Spd_Engine adjusted')
    df.Spd_Engine[0:start[0]] = df.Spd_Engine[start[0]]



In [ ]:
#find engine speeds tested

speeds = list(round(df.Spd_Engine[start]/100)*100) #take away noise from engine speeds

print(speeds)
fig, ax1 = plt.subplots()
ax1.plot(speeds, 'bo')
ax1.set_xlabel('sample')
ax1.set_ylim([500,2500])
ax1.set_yticks(np.arange(500,2750,250))

plt.show()

In [ ]:
(df.Spd_Engine[start])

In [ ]:
#smooth engine speed

rounded_speed = (round(df.Spd_Engine/10))*10
plt.plot(rounded_speed)
plt.show

# if engine speed during start is smaller than first LR speed

#fix_speed = (np.where(rounded_speed == speeds[0])[0])
#rounded_speed[0:max(fix_speed)] = speeds[0]
#np.shape(rounded_speed)

In [ ]:
# Define setspeed variable

S_Spd_Engine = (np.zeros((1, len(df.Spd_Engine))))
np.shape(S_Spd_Engine)

In [ ]:
# create setspeed variable & stop time for each LR

# create array where each LR stop
stop = list(np.zeros((len(speeds))))

for x in range(len(speeds)):
    filtered_speed =  list(np.where(rounded_speed == speeds[x])[0])
    S_Spd_Engine[0, min(filtered_speed):max(filtered_speed)] = speeds[x]
    
    stop[x] = max(list(np.where(rounded_speed == speeds[x])[0])) #create endpoint for each LR
    
plt.plot(S_Spd_Engine[0])
plt.show

In [ ]:
len(stop)

In [ ]:
len(start)

In [ ]:
len(speeds)

In [ ]:
b = 0
temp_start1 = np.zeros(round(len(start)/2))
temp_start2 = np.zeros(round(len(start)/2))

if (GTT == True):
    for i in range(round(len(start)/2)):
        temp_start1[i] = round(start[b])
        temp_start2[i] = round(stop[b])
        b += 2
    #start = np.append(temp_start1,temp_start2)
    start = temp_start1.tolist()
    stop = temp_start2.tolist()

In [ ]:
trqcalc = (np.zeros((len(speeds),14)))
trqcalc[:,0] = np.transpose(speeds)

for x in range(len(speeds)):
    #temp_trq = df.Trq_Engine[start[x]:stop[x]]
    trqcalc[x,1] = max(df.tc_TorqueValue[start[x]:stop[x]])
    trqcalc[x,2] = trqcalc[x,1]*0.9
    trqcalc[x,3] = trqcalc[x,1]*0.75
    trqcalc[x,4] = trqcalc[x,1]*0.5
    if (trqcalc[x,2] > max(df.Trq_Engine[start[x]:stop[x]])):
        trqcalc[x,5] = 'nan'
    else:
        trqcalc[x,5] = (min((np.where(df.Trq_Engine[start[x]:stop[x]] > trqcalc[x,2])[0]))-1)*dt
    trqcalc[x,6] = (min((np.where(df.Trq_Engine[start[x]:stop[x]] > trqcalc[x,3])[0]))-1)*dt
    trqcalc[x,7] = (min((np.where(df.Trq_Engine[start[x]:stop[x]] > trqcalc[x,4])[0]))-1)*dt
    if (trqcalc[x,2] > max(df.Trq_Engine[start[x]:stop[x]])):
        trqcalc[x,8] = 'nan'
    else:
        trqcalc[x,8] = linear_interp(df.Trq_Engine[start[x]:stop[x]], trqcalc[x,2])
    trqcalc[x,9] = linear_interp(df.Trq_Engine[start[x]:stop[x]], trqcalc[x,3])
    trqcalc[x,10] = linear_interp(df.Trq_Engine[start[x]:stop[x]], trqcalc[x,4])     
    trqcalc[x,11] = trqcalc[x,5]+trqcalc[x,8]
    trqcalc[x,12] = trqcalc[x,6]+trqcalc[x,9]
    trqcalc[x,13] = trqcalc[x,7]+trqcalc[x,10]

In [ ]:
# create matrix to data into
the_matrix = (np.zeros((len(speeds),21)))

# put in engine speed into the matrix column1
the_matrix[:,0] = np.transpose(speeds)
the_matrix[:,11:14] = trqcalc[:,11:14]

for x in range(len(speeds)):
    the_matrix[x,1] = df.Trq_Engine[start[x]] # trq at start of response
    the_matrix[x,2] = df.Trq_Engine[round(start[x]+0.3/dt)] # trq @0.3s
    the_matrix[x,3] = df.Trq_Engine[round(start[x]+0.5/dt)] # trq @0.5s
    the_matrix[x,4] = df.Trq_Engine[round(start[x]+1/dt)] # trq @1s
    the_matrix[x,5] = df.Trq_Engine[round(start[x]+1.5/dt)] #trq @1.5s
    the_matrix[x,6] = df.Trq_Engine[round(start[x]+2.0/dt)] #trq @2s
    the_matrix[x,7] = df.Trq_Engine[round(start[x]+2.5/dt)] #trq @2.5s
    the_matrix[x,8] = df.Trq_Engine[round(start[x]+3.0/dt)] #trq @3s
    the_matrix[x,9] = df.Trq_Engine[round(start[x]+4.0/dt)] #trq @4s
    the_matrix[x,10] = df.Trq_Engine[round(start[x]+15.0/dt)] #trq @15s
    the_matrix[x,14] = max(df.Conc_Soot483[start[x]:stop[x]]) #peak smoke
    the_matrix[x,15] = np.mean(df.Conc_Soot483[stop[x]-50:stop[x]]) #FL smoke
    the_matrix[x,16] = np.trapz(df.Conc_Soot483[start[x]:stop[x]])/100 #integrated smoke / 100 for plot fitting
    the_matrix[x,17] = max(df.Conc_NOx[start[x]:stop[x]]) #Peak NOx (rig)
    the_matrix[x,18] = max(df.se_NOxInLevel[start[x]:stop[x]]) #Peak NOx ECU
    the_matrix[x,19] = max(df.Spd_Turbo_rpm[start[x]:stop[x]]) #Max Turbo Speed
    the_matrix[x,20] = max(df.P_Cyl_Peak[start[x]:stop[x]]) #Max Turbo Speed

the_matrix = pd.DataFrame(data=the_matrix)

the_matrix_renamed = the_matrix.rename(columns={0: 'Engine_Speed',
                                                1: 'Trq_0s',
                                                2: 'Trq_0.3s',
                                                3: 'Trq_0.5s',
                                                4: 'Trq_1s',
                                                5:'Trq_1.5s',
                                                6:'Trq_2s',
                                                7:'Trq_2.5s',
                                                8:'Trq_3s',
                                                9:'Trq_4s',
                                                10:'Trq_15s',
                                                11:'t90%trq',
                                                12:'t75%trq',
                                                13:'t50%trq',
                                                14:'Peak_Smoke_483',
                                                15: 'FL_Smoke_483',
                                                16:'Integrated smoke',
                                                17:'Peak_Conc_NOx',
                                                18:'Peak_se_NOxInLevel',
                                                19:'Peak Turbospeed',
                                                20:'Peak Cyl Pres'})

the_matrix_dim = np.shape(the_matrix)
max_row = the_matrix_dim[0]
max_col = the_matrix_dim[1]-2



In [ ]:
## Create a Pandas Excel writer using XlsxWriter as the engine.
writefile = os.path.join(ResultPath,(str(os.path.splitext(FileNames[fileno])[0])+".xlsx"))
writer = pd.ExcelWriter(writefile, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
the_matrix_renamed.to_excel(writer, sheet_name='Response')
time2trq.to_excel(writer, sheet_name='Response', startcol=30)

workbook  = writer.book
worksheet = writer.sheets['Response']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '###0'})
format2 = workbook.add_format({'num_format': '#,##0.0'})


# Configure cells in worksheet
worksheet.set_column('B:B', 16, format1)
worksheet.set_column('C:Q', 16, format2)
worksheet.set_column('R:U', 16, format1)
worksheet.set_column('V:V', 16, format2)

# Create a chart object.
chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

# Configure the series of the chart from the dataframe data.

for i in range(10):
    col = 2 + i
    chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],        #'marker':     {'type': 'circle', 'size': 7, },
        
})

chart.set_title({'name': 'Torque after time'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2],
                  })
chart.set_y_axis({'name': 'Engine Torque (Nm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineTrq[0],
                  'max':EngineTrq[1],
                  'major_unit':EngineTrq[2],
                  'crossing':EngineTrq[0]})

chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('B22', chart, {'x_scale': 1.5, 'y_scale': 1.5})

# Create time2trq scatter plot

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

for i in range(3):
    col = 12 + i
    chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],    
})

# target times

for i in range(2):
    col = 32 + i
    chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 31, 4, 31],
            'values':     ['Response', 1, col,4, col],
            'marker': {'type': 'none'},
            'line': {
            'color': 'red',
            'width': 2,
            'dash_type': 'long_dash'}
            })  
    
chart.set_title({'name': 'Time to torque'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2]})
chart.set_y_axis({'name': 'Time (s)',
                  'major_gridlines': {'visible': True},
                  'min':0,
                  'max':5,
                  'major_unit':1})
chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('I22', chart, {'x_scale': 1.5, 'y_scale': 1.5})

# Create SMOKE scatter plot

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

for i in range(2):
    col = 15 + i
    chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],    
})

chart.add_series({
            'name':       ['Response', 0, 17],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, 17,max_row, 17],
            'marker': {'type': 'automatic'},
            'line': {
            'width': 2,
            'dash_type': 'long_dash'}
            })  

chart.set_title({'name': 'Smoke 439'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2]})

chart.set_y_axis({'name': 'Smoke 439 (mg/kg)',
                  'major_gridlines': {'visible': True},
                  'min':0,
                  'max':50,
                  'major_unit':5})



chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('B45', chart, {'x_scale': 1.5, 'y_scale': 1.5})


# Create Nox scatter plot

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

for i in range(2):
    col = 18 + i
    chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],    
})

chart.set_title({'name': 'NOx'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2]})
chart.set_y_axis({'name': 'NOx (ppm)',
                  'major_gridlines': {'visible': True},
                  'min':1000,
                  'max':1800,
                  'major_unit':100})

chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('I45', chart, {'x_scale': 1.5, 'y_scale': 1.5})

# Create Turbospeed Scatter Plot

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

col = 20
chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],    
})

chart.set_title({'name': 'Turbo Speed'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2]})
chart.set_y_axis({'name': 'Turbo Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':TurboSpeed[0],
                  'max':TurboSpeed[1],
                  'major_unit':TurboSpeed[2]})

chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('P22', chart, {'x_scale': 1.5, 'y_scale': 1.5})

# Create P_Cyl_Peak Scatter Plot

chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

col = 21
chart.add_series({
            'name':       ['Response', 0, col],
            'categories': ['Response', 1, 1, max_row, 1],
            'values':     ['Response', 1, col, max_row, col],    
})

chart.set_title({'name': 'Peak Cylinder Pressure'})
chart.set_x_axis({'name': 'Engine Speed (rpm)',
                  'major_gridlines': {'visible': True},
                  'min':EngineSpeed[0],
                  'max':EngineSpeed[1],
                  'major_unit':EngineSpeed[2]})
chart.set_y_axis({'name': 'Peak Cylinder Pressure (MPa)',
                  'major_gridlines': {'visible': True},
                  'min':Pcyl[0],
                  'max':Pcyl[1],
                  'major_unit':Pcyl[2]})

chart.set_legend({
    'position': 'right',})

# Insert the chart into the worksheet.
worksheet.insert_chart('P45', chart, {'x_scale': 1.5, 'y_scale': 1.5})


# Close the Pandas Excel writer and output the Excel file.
writer.save()    

In [ ]:
print ('The script took {0} second !'.format(time.time() - startTime))